In [1]:
import numpy as np
import pandas as pd
import warnings
import gc
import matplotlib.pyplot as plt
import datetime
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
warnings.filterwarnings('ignore')
plt.style.use('seaborn')

In [2]:
# Reduce the memory usage - Inspired by Panchajanya Banerjee
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return df


In [3]:
train = reduce_mem_usage(pd.read_csv('../data/train.csv', parse_dates=["first_active_month"]))
test = reduce_mem_usage(pd.read_csv('../data/test.csv', parse_dates=["first_active_month"]))

# Now extract the month, year, day, weekday
train["year"] = train["first_active_month"].dt.year
train["month"] = train["first_active_month"].dt.month
train["dayofyear"] = train["first_active_month"].dt.dayofyear
train['week'] = train["first_active_month"].dt.weekofyear
train['dayofweek'] = train['first_active_month'].dt.dayofweek
train['days'] = (datetime.date(2018, 2, 1) - train['first_active_month'].dt.date).dt.days

test["year"] = test["first_active_month"].dt.year
test["month"] = test["first_active_month"].dt.month
test["dayofyear"] = test["first_active_month"].dt.dayofyear
test['week'] = test["first_active_month"].dt.weekofyear
test['dayofweek'] = test['first_active_month'].dt.dayofweek
test['days'] = (datetime.date(2018, 2, 1) - test['first_active_month'].dt.date).dt.days

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)


In [4]:
# # Taking Reference from Other Kernels
# def aggregate_transaction_hist(trans, prefix):
#     agg_func = {
#         'purchase_date': ['max', 'min'],
#         'month_diff': ['mean'],
#         'weekend': ['sum', 'mean'],
#         'authorized_flag': ['sum', 'mean'],
#         'category_1': ['sum', 'mean'],
#         'purchase_amount': ['sum', 'mean', 'count', 'max', 'min', 'std'],
#         'installments': ['sum', 'mean', 'count', 'max', 'min', 'std'],
#         'merchant_id': ['nunique'],
#         'month_lag': ['max', 'min', 'mean', 'var'],
#         'month_diff': ['mean'],
#         'card_id': ['size'],
#         'month': ['nunique'],
#         'hour': ['nunique'],
#         'weekofyear': ['nunique'],
#         'dayofweek': ['nunique'],
#         'year': ['nunique'],
#         'subsector_id': ['nunique'],
#         'merchant_category_id': ['nunique'],
#         'Christmas_Day_2017': ['mean'],
#         'Mothers_Day_2017':['mean'],
#         'fathers_day_2017': ['mean'],
#         'Children_day_2017': ['mean'],
#         'Black_Friday_2017': ['mean'],
#         'Valentine_day_2017': ['mean'],
#         'Mothers_Day_2018': ['mean']
#     }

#     agg_trans = trans.groupby(['card_id']).agg(agg_func)
#     agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
#     agg_trans.reset_index(inplace=True)
#     df = (trans.groupby('card_id').size().reset_index(name='{}transactions_count'.format(prefix)))
#     agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')

#     return agg_trans

# Taking Reference from Other Kernels
def aggregate_transaction_hist(trans, prefix):
    agg_func = {'purchase_date': ['max', 'min'],
                'month_diff': ['mean'],
                'weekend': ['sum', 'mean'],
                'authorized_flag': ['sum', 'mean'],
                'category_1': ['sum', 'mean'],
                'purchase_amount': ['sum', 'mean', 'count', 'max', 'min', 'std'],
                'installments': ['sum', 'mean', 'count', 'max', 'min', 'std'],
                'merchant_id': ['nunique'],
                'month_lag': ['max', 'min', 'mean', 'var'],
                'month_diff': ['mean'],
                'card_id': ['size'],
                'month': ['nunique'],
                'hour': ['nunique'],
                'weekofyear': ['nunique'],
                'dayofweek': ['nunique'],
                'year': ['nunique'],
                'subsector_id': ['nunique'],
                'merchant_category_id': ['nunique'],
                'Christmas_Day_2017': ['mean'],
                'Mothers_Day_2017':['mean'],
                'fathers_day_2017': ['mean'],
                'Children_day_2017': ['mean'],
                'Black_Friday_2017': ['mean'],
                'Valentine_day_2017': ['mean'],
                'Mothers_Day_2018': ['mean']}
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    df = (trans.groupby('card_id').size().reset_index(name='{}transactions_count'.format(prefix)))
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')

    return agg_trans


In [5]:
transactions = reduce_mem_usage(pd.read_csv('../data/historical_transactions.csv'))
transactions['authorized_flag'] = transactions['authorized_flag'].map({'Y': 1, 'N': 0})
transactions['category_1'] = transactions['category_1'].map({'Y': 1, 'N': 0})

# Feature Engineering - Adding new features inspired by Chau's first kernel
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])
transactions['year'] = transactions['purchase_date'].dt.year
transactions['weekofyear'] = transactions['purchase_date'].dt.weekofyear
transactions['month'] = transactions['purchase_date'].dt.month
transactions['dayofweek'] = transactions['purchase_date'].dt.dayofweek
transactions['weekend'] = (transactions.purchase_date.dt.weekday >= 5).astype(int)
transactions['hour'] = transactions['purchase_date'].dt.hour
transactions['month_diff'] = ((datetime.datetime.today() - transactions['purchase_date']).dt.days) // 30
transactions['month_diff'] += transactions['month_lag']

# impute missing values - This is now excluded.
transactions['category_2'] = transactions['category_2'].fillna(1.0, inplace=True)
transactions['category_3'] = transactions['category_3'].fillna('A', inplace=True)
transactions['merchant_id'] = transactions['merchant_id'].fillna('M_ID_00a6ca8a8a', inplace=True)
gc.collect()

Mem. usage decreased to 1749.11 Mb (43.7% reduction)


185

In [6]:
# agg_func = {'mean': ['mean']}
# for col in ['category_2', 'category_3']:
#     transactions[col + '_mean'] = transactions['purchase_amount'].groupby(transactions[col]).agg(agg_func)

# transactions['Christmas_Day_2017'] = (pd.to_datetime('2017-12-25') - transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # Mothers Day: May 14 2017
# transactions['Mothers_Day_2017'] = (pd.to_datetime('2017-05-04') - transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # fathers day: August 13 2017
# transactions['fathers_day_2017'] = (pd.to_datetime('2017-08-13') - transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # Childrens day: October 12 2017
# transactions['Children_day_2017'] = (pd.to_datetime('2017-10-12') - transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # Black Friday : 24th November 2017
# transactions['Black_Friday_2017'] = (pd.to_datetime('2017-11-24') - transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # Valentines Day
# transactions['Valentine_day_2017'] = (pd.to_datetime('2017-06-12') - transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)

# # 2018
# # Mothers Day: May 13 2018
# transactions['Mothers_Day_2018'] = (pd.to_datetime('2018-05-13') - transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)

# gc.collect()

# merge_trans = aggregate_transaction_hist(transactions, prefix='hist_')
# del transactions
# gc.collect()
# train = pd.merge(train, merge_trans, on='card_id', how='left')
# test = pd.merge(test, merge_trans, on='card_id', how='left')
# del merge_trans
# gc.collect()


agg_func = {'mean': ['mean']}
for col in ['category_2', 'category_3']:
    transactions[col + '_mean'] = transactions['purchase_amount'].groupby(transactions[col]).agg(agg_func)

# Christmas Eve
transactions['Christmas_Eva_2017'] = (pd.to_datetime('2017-12-24') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)    
# Christmas Day：12 25 2017 
transactions['Christmas_Day_2017'] = (pd.to_datetime('2017-12-25') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Mothers Day: May 14 2017
transactions['Mothers_Day_2017'] = (pd.to_datetime('2017-05-04') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# fathers day: August 13 2017
transactions['fathers_day_2017'] = (pd.to_datetime('2017-08-13') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Childrens day: October 12 2017
transactions['Children_day_2017'] = (pd.to_datetime('2017-10-12') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Black Friday : 24th November 2017
transactions['Black_Friday_2017'] = (pd.to_datetime('2017-11-24') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Valentines Day
transactions['Valentine_day_2017'] = (pd.to_datetime('2017-06-12') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Global Shopping ：11-11
transactions['Global_Shopping_2017'] = (pd.to_datetime('2017-11-11') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# New Year ：1-1-2018
transactions['New_year_2017'] = (pd.to_datetime('2017-01-01') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Independence_Day: 7-4-2017
transactions['Independence_day_2017'] = (pd.to_datetime('2017-07-04') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Valentine_Day: 2-14-2017
transactions['Valentine’s_day_2017'] = (pd.to_datetime('2017-02-14') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Halloween_Day: 10-30-2017
transactions['Halloween_day_2017'] = (pd.to_datetime('2017-10-30') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Mothers Day: May 13 2018
transactions['Mothers_Day_2018'] = (pd.to_datetime('2018-05-13') - transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)

gc.collect()

merge_trans = aggregate_transaction_hist(transactions, prefix='hist_')
del transactions
gc.collect()
# data = pd.merge(data, merge_trans, on='card_id', how='left')

train = pd.merge(train, merge_trans, on='card_id', how='left')
test = pd.merge(test, merge_trans, on='card_id', how='left')
del merge_trans
gc.collect()

49

In [7]:
# Feature Engineering - Adding new features inspired by Chau's first kernel
train['hist_purchase_date_max'] = pd.to_datetime(train['hist_purchase_date_max'])
train['hist_purchase_date_min'] = pd.to_datetime(train['hist_purchase_date_min'])
train['hist_purchase_date_diff'] = (train['hist_purchase_date_max'] - train['hist_purchase_date_min']).dt.days
train['hist_purchase_date_average'] = train['hist_purchase_date_diff'] / train['hist_card_id_size']
train['hist_purchase_date_uptonow'] = (datetime.datetime.today() - train['hist_purchase_date_max']).dt.days
train['hist_first_buy'] = (train['hist_purchase_date_min'] - train['first_active_month']).dt.days
for feature in ['hist_purchase_date_max', 'hist_purchase_date_min']:
    train[feature] = train[feature].astype(np.int64) * 1e-9

# Feature Engineering - Adding new features inspired by Chau's first kernel
test['hist_purchase_date_max'] = pd.to_datetime(test['hist_purchase_date_max'])
test['hist_purchase_date_min'] = pd.to_datetime(test['hist_purchase_date_min'])
test['hist_purchase_date_diff'] = (test['hist_purchase_date_max'] - test['hist_purchase_date_min']).dt.days
test['hist_purchase_date_average'] = test['hist_purchase_date_diff'] / test['hist_card_id_size']
test['hist_purchase_date_uptonow'] = (datetime.datetime.today() - test['hist_purchase_date_max']).dt.days
test['hist_first_buy'] = (test['hist_purchase_date_min'] - test['first_active_month']).dt.days
for feature in ['hist_purchase_date_max', 'hist_purchase_date_min']:
    test[feature] = test[feature].astype(np.int64) * 1e-9

In [8]:
# # Taking Reference from Other Kernels
# def aggregate_transaction_new(trans, prefix):
#     agg_func = {
#         'purchase_date': ['max', 'min'],
#         'month_diff': ['mean'],
#         'weekend': ['sum', 'mean'],
#         'authorized_flag': ['sum'],
#         'category_1': ['sum', 'mean'],
#         'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
#         'installments': ['sum', 'mean', 'max', 'min', 'std'],
#         'merchant_id': ['nunique'],
#         'month_lag': ['max', 'min', 'mean', 'var'],
#         'month_diff': ['mean'],
#         'card_id': ['size'],
#         'month': ['nunique'],
#         'hour': ['nunique'],
#         'weekofyear': ['nunique'],
#         'dayofweek': ['nunique'],
#         'year': ['nunique'],
#         'subsector_id': ['nunique'],
#         'merchant_category_id': ['nunique'],
#         'Christmas_Day_2017': ['mean'],
#         'Mothers_Day_2017':['mean'],
#         'fathers_day_2017': ['mean'],
#         'Children_day_2017': ['mean'],
#         'Black_Friday_2017': ['mean'],
#         'Valentine_Day_2017': ['mean'],
#         'Mothers_Day_2018': ['mean']
#     }

#     agg_trans = trans.groupby(['card_id']).agg(agg_func)
#     agg_trans.columns = [prefix + '_'.join(col).strip()
#                          for col in agg_trans.columns.values]
#     agg_trans.reset_index(inplace=True)

#     df = (trans.groupby('card_id')
#           .size()
#           .reset_index(name='{}transactions_count'.format(prefix)))

#     agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')

#     return agg_trans


# Taking Reference from Other Kernels
def aggregate_transaction_new(trans, prefix):
    agg_func = {"purchase_date":["max", "min"],
                "month_diff":["mean"],
                "weekend": ["sum", "mean"],
                "authorized_flag": ["sum"],
                "category_1": ["sum", "mean"],
                "purchase_amount": ["sum", "mean", "max", "min", "std", "var", "count", "median"],
                "installments": ["sum", "mean", "max", "min", "std", "var", "count", "median"],
                "merchant_id": ["nunique"],
                "month_lag": ["max", "min", "mean", "var", "median", "sum", "count", "std"],
                "month_diff": ["mean"],
                "card_id": ["size"],
                "month": ["nunique"],
                "hour": ["nunique"],
                "weekofyear": ["nunique"],
                "dayofweek": ["nunique"],
                "year": ["nunique"],
                "subsector_id": ["nunique"],
                "merchant_category_id": ["nunique"],
                "Christmas_Day_2017": ["mean"],
                "Mothers_Day_2017": ["mean"],
                "fathers_day_2017": ["mean"],
                "Children_day_2017": ["mean"],
                "Black_Friday_2017": ["mean"],
                "Valentine_Day_2017": ["mean"],
                "Christmas_Eva_2017": ["mean"],
                "Global_Shopping_2017": ["mean"],
                "New_year_2017": ["mean"],
                "Independence_day_2017": ["mean"],
                "Valentine’s_day_2017": ["mean"],
                "Halloween_day_2017": ["mean"],
                "Mothers_Day_2018": ["mean"]
                }
    agg_trans = trans.groupby(["card_id"]).agg(agg_func)
    agg_trans.columns = [prefix + "_".join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    df = (trans.groupby("card_id").size().reset_index(name="{}transactions_count".format(prefix)))
    agg_trans = pd.merge(df, agg_trans, on="card_id", how="left")
    
    return agg_trans


In [9]:
# Now extract the data from the new transactions
new_transactions = reduce_mem_usage(pd.read_csv('../data/new_merchant_transactions.csv'))
new_transactions['authorized_flag'] = new_transactions['authorized_flag'].map({'Y': 1, 'N': 0})
new_transactions['category_1'] = new_transactions['category_1'].map({'Y': 1, 'N': 0})

# Feature Engineering - Adding new features inspired by Chau's first kernel
new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])
new_transactions['year'] = new_transactions['purchase_date'].dt.year
new_transactions['weekofyear'] = new_transactions['purchase_date'].dt.weekofyear
new_transactions['month'] = new_transactions['purchase_date'].dt.month
new_transactions['dayofweek'] = new_transactions['purchase_date'].dt.dayofweek
new_transactions['weekend'] = (new_transactions.purchase_date.dt.weekday >= 5).astype(int)
new_transactions['hour'] = new_transactions['purchase_date'].dt.hour
new_transactions['month_diff'] = ((datetime.datetime.today() - new_transactions['purchase_date']).dt.days) // 30
new_transactions['month_diff'] += new_transactions['month_lag']

# impute missing values
new_transactions['category_2'] = new_transactions['category_2'].fillna(1.0, inplace=True)
new_transactions['category_3'] = new_transactions['category_3'].fillna('A', inplace=True)
new_transactions['merchant_id'] = new_transactions['merchant_id'].fillna('M_ID_00a6ca8a8a', inplace=True)

Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [10]:
# # New Features with Key Shopping times considered in the dataset. if the purchase has been made within 60 days, 
# # it is considered as an influence

# # Christmas : December 25 2017
# new_transactions['Christmas_Day_2017'] = (
#             pd.to_datetime('2017-12-25') - new_transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # Mothers Day: May 14 2017 - Was not significant in Feature Importance
# new_transactions['Mothers_Day_2017'] = (pd.to_datetime('2017-06-04') - new_transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # fathers day: August 13 2017
# new_transactions['fathers_day_2017'] = (pd.to_datetime('2017-08-13') - new_transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # Childrens day: October 12 2017
# new_transactions['Children_day_2017'] = (
#             pd.to_datetime('2017-10-12') - new_transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # Valentine's Day : 12th June, 2017
# new_transactions['Valentine_Day_2017'] = (
#             pd.to_datetime('2017-06-12') - new_transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)
# # Black Friday : 24th November 2017
# new_transactions['Black_Friday_2017'] = (
#             pd.to_datetime('2017-11-24') - new_transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)

# # 2018
# # Mothers Day: May 13 2018
# new_transactions['Mothers_Day_2018'] = (pd.to_datetime('2018-05-13') - new_transactions['purchase_date']).dt.days.apply(
#     lambda x: x if x > 0 and x < 100 else 0)

# agg_func = {'mean': ['mean']}
# for col in ['category_2', 'category_3']:
#     new_transactions[col + '_mean'] = new_transactions['purchase_amount'].groupby(new_transactions[col]).agg(agg_func)

# gc.collect()

# merge_new = aggregate_transaction_new(new_transactions, prefix='new_')
# del new_transactions
# gc.collect()

# train = pd.merge(train, merge_new, on='card_id', how='left')
# test = pd.merge(test, merge_new, on='card_id', how='left')
# del merge_new

# gc.collect()


# New Features with Key Shopping times considered in the dataset. if the purchase has been made within 60 days, 
# it is considered as an influence
agg_func = {'mean': ['mean']}
for col in ['category_2', 'category_3']:
    new_transactions[col + '_mean'] = new_transactions['purchase_amount'].groupby(new_transactions[col]).agg(agg_func)

# Christmas Eve
new_transactions['Christmas_Eva_2017'] = (pd.to_datetime('2017-12-24') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)    
# Christmas Day：12 25 2017 
new_transactions['Christmas_Day_2017'] = (pd.to_datetime('2017-12-25') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Mothers Day: May 14 2017
new_transactions['Mothers_Day_2017'] = (pd.to_datetime('2017-05-04') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# fathers day: August 13 2017
new_transactions['fathers_day_2017'] = (pd.to_datetime('2017-08-13') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Childrens day: October 12 2017
new_transactions['Children_day_2017'] = (pd.to_datetime('2017-10-12') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Black Friday : 24th November 2017
new_transactions['Black_Friday_2017'] = (pd.to_datetime('2017-11-24') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Valentines Day
new_transactions['Valentine_Day_2017'] = (pd.to_datetime('2017-06-12') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Global Shopping ：11-11
new_transactions['Global_Shopping_2017'] = (pd.to_datetime('2017-11-11') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# New Year ：1-1-2018
new_transactions['New_year_2017'] = (pd.to_datetime('2017-01-01') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Independence_Day: 7-4-2017
new_transactions['Independence_day_2017'] = (pd.to_datetime('2017-07-04') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Valentine_Day: 2-14-2017
new_transactions['Valentine’s_day_2017'] = (pd.to_datetime('2017-02-14') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Halloween_Day: 10-30-2017
new_transactions['Halloween_day_2017'] = (pd.to_datetime('2017-10-30') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)
# Mothers Day: May 13 2018
new_transactions['Mothers_Day_2018'] = (pd.to_datetime('2018-05-13') - new_transactions['purchase_date']).dt.days.apply(
    lambda x: x if x > 0 and x < 100 else 0)

gc.collect()

merge_new = aggregate_transaction_new(new_transactions, prefix='new_')
del new_transactions
gc.collect()
# data = pd.merge(data, merge_new, on='card_id', how='left')

train = pd.merge(train, merge_new, on='card_id', how='left')
test = pd.merge(test, merge_new, on='card_id', how='left')
del merge_new
gc.collect()


35

In [11]:
# Feature Engineering - Adding new features inspired by Chau's first kernel
train['new_purchase_date_max'] = pd.to_datetime(train['new_purchase_date_max'])
train['new_purchase_date_min'] = pd.to_datetime(train['new_purchase_date_min'])
train['new_purchase_date_diff'] = (train['new_purchase_date_max'] - train['new_purchase_date_min']).dt.days
train['new_purchase_date_average'] = train['new_purchase_date_diff'] / train['new_card_id_size']
train['new_purchase_date_uptonow'] = (datetime.datetime.today() - train['new_purchase_date_max']).dt.days
train['new_first_buy'] = (train['new_purchase_date_min'] - train['first_active_month']).dt.days
for feature in ['new_purchase_date_max', 'new_purchase_date_min']:
    train[feature] = train[feature].astype(np.int64) * 1e-9

# Feature Engineering - Adding new features inspired by Chau's first kernel
test['new_purchase_date_max'] = pd.to_datetime(test['new_purchase_date_max'])
test['new_purchase_date_min'] = pd.to_datetime(test['new_purchase_date_min'])
test['new_purchase_date_diff'] = (test['new_purchase_date_max'] - test['new_purchase_date_min']).dt.days
test['new_purchase_date_average'] = test['new_purchase_date_diff'] / test['new_card_id_size']
test['new_purchase_date_uptonow'] = (datetime.datetime.today() - test['new_purchase_date_max']).dt.days
test['new_first_buy'] = (test['new_purchase_date_min'] - test['first_active_month']).dt.days
for feature in ['new_purchase_date_max', 'new_purchase_date_min']:
    test[feature] = test[feature].astype(np.int64) * 1e-9

# added new feature - Interactive
train['card_id_total'] = train['new_card_id_size'] + train['hist_card_id_size']
train['purchase_amount_total'] = train['new_purchase_amount_sum'] + train['hist_purchase_amount_sum']

test['card_id_total'] = test['new_card_id_size'] + test['hist_card_id_size']
test['purchase_amount_total'] = test['new_purchase_amount_sum'] + test['hist_purchase_amount_sum']

gc.collect()


79

In [12]:
# Check for missing values in training set
nulls = np.sum(train.isnull())
nullcols = nulls.loc[(nulls != 0)]
dtypes = train.dtypes
dtypes2 = dtypes.loc[(nulls != 0)]
info = pd.concat([nullcols, dtypes2], axis=1).sort_values(by=0, ascending=False)

# Check for missing values in training set
nulls = np.sum(test.isnull())
nullcols = nulls.loc[(nulls != 0)]
dtypes = test.dtypes
dtypes2 = dtypes.loc[(nulls != 0)]
info = pd.concat([nullcols, dtypes2], axis=1).sort_values(by=0, ascending=False)

numeric_dtypes = ['float64']
numerics = []
for i in train.columns:
    if train[i].dtype in numeric_dtypes:
        numerics.append(i)

numeric_dtypes = ['float64']
numerics = []
for i in test.columns:
    if test[i].dtype in numeric_dtypes:
        numerics.append(i)

train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

for features in ['feature_1', 'feature_2', 'feature_3']:
    order_label = train.groupby([features])['outliers'].mean()
    train[features] = train[features].map(order_label)
    test[features] = test[features].map(order_label)

In [13]:
df_train_columns = [c for c in train.columns if c not in ['card_id', 'first_active_month', 'target', 'outliers']]
target = train['target']

param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.001,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": -1}

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])

    num_round = 20000
    clf = lgb.train(param, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=-1,
                    early_stopping_rounds=100)
    oof[val_idx] = clf.predict(train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions += clf.predict(test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

sample_submission = pd.read_csv('../data/sample_submission.csv')
sample_submission['target'] = predictions
sample_submission.to_csv('../submission/submission_lgb20000fujia.csv', index=False)

fold 0
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[9571]	training's rmse: 3.37726	valid_1's rmse: 3.66105
fold 1
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[8500]	training's rmse: 3.39652	valid_1's rmse: 3.66607
fold 2
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[10175]	training's rmse: 3.37314	valid_1's rmse: 3.64758
fold 3
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[10580]	training's rmse: 3.35826	valid_1's rmse: 3.65056
fold 4
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[10519]	training's rmse: 3.36647	valid_1's rmse: 3.65551
